# Automated Financial Research & Analysis using Swarm of agents with Strands Agents

## Understanding Multi-Agent Systems and Swarm Intelligence

An agent swarm is a collection of autonomous AI agents working together to solve complex problems through collaboration. Inspired by natural systems like ant colonies or bird flocks, agent swarms leverage collective intelligence where the combined output exceeds what any single agent could produce. By distributing tasks and sharing information, swarms can tackle complex problems more efficiently and effectively than individual agents working in isolation.

Multi-agent systems consist of multiple interacting intelligent agents within an environment. These systems enable:

- *Distributed Problem Solving*: Breaking complex tasks into subtasks for parallel processing
- *Information Sharing*: Agents exchange insights to build collective knowledge
- *Specialization*: Different agents focus on specific aspects of a problem
- *Redundancy*: Multiple agents working on similar tasks improve reliability
- *Emergent Intelligence*: The system exhibits capabilities beyond those of its individual components

Swarm intelligence emphasizes:

1. *Decentralized Control*: No single agent directs the entire system
2. *Local Interactions*: Agents primarily interact with nearby agents
3. *Simple Rules*: Individual agents follow relatively simple behaviors
4. *Emergent Complexity*: Complex system behavior emerges from simple agent interactions

## Creating a Swarm with Strands Agents

### Automated Financial Research & Analysis
Let's explore a Swarm of specialized agents using mesh communication to solve problems collaboratively
This mesh architecture enables direct communication between all agents, allowing each agent to share insights with every other agent. The specialized roles (research, investment, risk, and summarizer) work together to produce a comprehensive solution that benefits from multiple perspectives and iterative refinement.

An agent that can ingest a financial report, news and analyzes market data from multiple sources. A creative agent can be used to evaluate new inverstment opportunities. Another agent evaluates risks, and an editorial agent generate a summary report with key insights.

Let's see how to build this Pattern —> Mesh



###  Mesh Swarm Architecture

Strands Agents SDK allows you to create Swarms using existing Agent objects, even when they use different model providers or have different configurations. While various communication architectures are possible (hierarchical, parallel, sequential, and mesh), the following example demonstrates a mesh architecture implementation, which provides a flexible foundation for agent-to-agent communication.



<p align="center">
    <img src="./images/Mesh_swarm.png">
</p>

## 1. Quick Start with Swarm tool

The Strands Agents SDK provides a built-in swarm tool that simplifies the implementation of multi-agent systems, offering a quick start for users. This tool implements the shared memory.

In [17]:
!pip install -r requirements.txt
!pip install PyPDF2

In [23]:
from strands import Agent
from strands_tools import swarm
import time
import PyPDF2
import json
import boto3
import yaml
import os
import logging


In [24]:
import PyPDF2

def read_pdf(file_path):
    """
    Read and extract text from a PDF file.
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted text from the PDF
    """
    try:
        # Open the PDF file in binary read mode
        with open(file_path, 'rb') as file:
            # Create a PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Get number of pages
            num_pages = len(pdf_reader.pages)
            print(f"Total pages: {num_pages}")
            
            # Extract text from each page
            text = ""
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
                
            return text
    
    except FileNotFoundError:
        return "Error: The file was not found."
    except PyPDF2.errors.PdfReadError:
        return "Error: Invalid PDF file or the file is encrypted."
    except Exception as e:
        return f"Error: {str(e)}"



In [25]:
# Example usage
if __name__ == "__main__":
    pdf_path = "data/amzn-20241231-10K-Part-1&2.pdf"  # Replace with your PDF file path
    financial_report = read_pdf(pdf_path)
    print("financial_report text:")
    print(financial_report[0:1000])

Total pages: 39
financial_report text:
Table of ContentsAMAZON.COM, INC.PART IItem 1.BusinessThis Annual Report on Form 10-K and the documents incorporated herein by reference contain forward-looking statementsbased on expectations, estimates, and projections as of the date of this filing. Actual results and outcomes may differ materially fromthose expressed in forward-looking statements. See Item 1A of Part I — “Risk Factors.” As used herein, “Amazon.com,” “we,” “our,”and similar terms include Amazon.com, Inc. and its subsidiaries, unless the context indicates otherwise.GeneralWe seek to be Earth’s most customer-centric company. We are guided by four principles: customer obsession rather thancompetitor focus, passion for invention, commitment to operational excellence, and long-term thinking. In each of our segments, weserve our primary customer sets, consisting of consumers, sellers, developers, enterprises, content creators, advertisers, and employees.We have organized our operation

In a mesh architecture, all agents can communicate directly with each other. The following example demonstrates a swarm of specialized agents using mesh communication to solve problems collaboratively:

In [26]:
# Create specialized agents with different expertise
research_agent = Agent(system_prompt=("""You are a Financial Research Agent specializing in gathering and analyzing information.
Your role in the swarm is to provide factual information and research insights on the topic.
You should focus on providing accurate data and identifying key aspects of the problem.
When receiving input from other agents, evaluate if their information aligns with your research.
"""), 
callback_handler=None)

investment_agent = Agent(system_prompt=("""You are a Investmen Agent specializing in evaluating innovative approach to new investments.
Your role in the swarm is to think outside the box and propose creative investment approaches.
You should build upon information from other agents while adding your unique creative perspective.
Focus on novel approaches that others might not have considered.
"""), 
callback_handler=None)

risk_agent = Agent(system_prompt=("""You are a Critical Agent specializing in analyzing investment proposals and finding flaws and risks.
Your role in the swarm is to evaluate investment proposed by other agents and identify potential issues.
You should carefully examine proposed investments, find weaknesses or oversights, and suggest improvements.
Be constructive in your criticism while ensuring the final solution is robust.
"""), 
callback_handler=None)

summarizer_agent = Agent(system_prompt="""You are a Summarizer Agent specializing in synthesizing information.
Your role in the swarm is to gather insights from all agents and create a cohesive final solution.
You should combine the best ideas and address the criticisms to create a comprehensive response.
Focus on creating a clear, actionable summary that addresses the original query effectively.
""")

The mesh communication is implemented using a dictionary to track messages between agents:

In [27]:
# Dictionary to track messages between agents (mesh communication)
messages = {
    "research": [],
    "investment": [],
    "risk": [],
    "summarizer": []
}

The swarm operates in multiple phases, with each agent first analyzing the problem independently:

In [28]:
query = f"""
             Analyze the 10K Financial report in {financial_report} and tell me if this is a good company to invest
             """

In [29]:
# Phase 1: Initial analysis by each specialized agent
research_result = research_agent(query)
time.sleep(30)
investment_result = investment_agent(query)
time.sleep(30)
risk_result = risk_agent(query)

After the initial analysis, results are shared with all other agents (mesh communication):

In [30]:
# Share results with all other agents (mesh communication)
messages["investment"].append(f"From Research Agent: {research_result}")
messages["risk"].append(f"From Research Agent: {research_result}")
messages["summarizer"].append(f"From Research Agent: {research_result}")

messages["research"].append(f"From Creative Agent: {investment_result}")
messages["risk"].append(f"From Creative Agent: {investment_result}")
messages["summarizer"].append(f"From Creative Agent: {investment_result}")

messages["research"].append(f"From Critical Agent: {risk_result}")
messages["investment"].append(f"From Critical Agent: {risk_result}")
messages["summarizer"].append(f"From Critical Agent: {risk_result}")

In the second phase, each agent refines their solution based on input from all other agents:

In [31]:
# Phase 2: Each agent refines based on input from others
research_prompt = f"{query}\n\nConsider these messages from other agents:\n" + "\n\n".join(messages["research"])
investment_prompt = f"{query}\n\nConsider these messages from other agents:\n" + "\n\n".join(messages["investment"])
risk_prompt = f"{query}\n\nConsider these messages from other agents:\n" + "\n\n".join(messages["risk"])

refined_research = research_agent(research_prompt)

time.sleep(30)

refined_investment = investment_agent(investment_prompt)

time.sleep(30)

refined_risk = risk_agent(risk_prompt)

# Share refined results with summarizer
messages["summarizer"].append(f"From Research Agent (Phase 2): {refined_research}")
messages["summarizer"].append(f"From Investment Agent (Phase 2): {refined_investment}")
messages["summarizer"].append(f"From Critical Agent (Phase 2): {refined_risk}")

Finally, the summarizer agent synthesizes all inputs into a comprehensive solution:

In [32]:
# Final phase: Summarizer creates the final solution
time.sleep(30)

summarizer_prompt = f"""
Original query: {query}

Please synthesize the following inputs from all agents into a comprehensive final solution:

{"\n\n".join(messages["summarizer"])}

Create a well-structured final answer that incorporates the research findings, 
creative ideas, and addresses the critical feedback.
Provide recommendation weather to invest in this company or not.Provide rationale to support your recommendation
"""

final_solution = str(summarizer_agent(summarizer_prompt))

# Amazon (AMZN) Investment Analysis: Should You Invest?

## Executive Summary

Based on a comprehensive analysis of Amazon's 2024 10-K financial report, Amazon represents a compelling investment opportunity for long-term investors who can tolerate some volatility. The company demonstrates strong revenue growth across diverse business segments, exceptional cash flow generation, strategic investments in high-growth sectors like cloud computing and artificial intelligence, and a proven track record of innovation. However, investors should be mindful of significant capital expenditure requirements, regulatory risks, and intense competition across all business segments.

## Financial Performance Overview

Amazon's financial performance remains robust:

- **Revenue Growth**: 11% year-over-year increase to $637.96 billion in 2024
- **Segment Performance**:
  - North America: $387.50 billion (10% growth), 61% of total revenue
  - International: $142.91 billion (9% growth), 22% of total revenue

In [ ]:
print(final_solution)

This mesh architecture enables direct communication between all agents, allowing each agent to share insights with every other agent. The specialized roles (research, creative, critical, and summarizer) work together to produce a comprehensive solution that benefits from multiple perspectives and iterative refinement.

### 2.2 Implementing Shared Memory

While the mesh communication example effectively demonstrates agent collaboration, a shared memory system would enhance the swarm's capabilities by providing:

- A centralized knowledge repository for all agents
- Automated phase tracking and historical knowledge preservation
- Thread-safe concurrent access for improved efficiency
- Persistent storage of insights across multiple interactions

Extending our mesh swarm example with shared memory would replace the message dictionary with a SharedMemory instance, simplifying the code while enabling more sophisticated knowledge management.

## When to use Swarm:

- For quick, parallel processing of a single complex task
- When you need multiple perspectives on the same problem
- For tasks that benefit from collective intelligence

## Conclusion

Multi-agent swarms solve complex problems through collective intelligence. The Strands Agents SDK supports both custom implementations and a built-in swarm tool with shared memory. By distributing tasks across specialized agents and enabling effective communication, swarms achieve better results than single agents working alone. Whether using mesh communication patterns or the swarm tool, developers can create systems where multiple agents work together with defined roles, coordination mechanisms, and knowledge sharing.

